In [ ]:
import os
import pydicom
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import matplotlib.pyplot as plt
from pydicom.pixels import iter_pixels

class DICOMClassificationModel:
    def __init__(self, img_height=224, img_width=224):
        """
        DICOM görüntüleri için VGG16 sınıflandırma modeli
        
        Args:
            img_height (int): Görüntü yüksekliği
            img_width (int): Görüntü genişliği
        """
        self.img_height = img_height
        self.img_width = img_width
        self.classes = ['Tcga_lgg_astro_filtered', 'Tcga_gbm_filtered', 'Tcga_lgg_olio_filtered']
        self.model = self.build_model()
    
    def load_dicom_images(self, base_dir):
        """
        DICOM dosyalarını yükleme ve sınıflandırma
        
        Args:
            base_dir (str): DICOM dosyaları içeren dizin
        
        Returns:
            tuple: Görüntüler ve etiketler
        """
        images = []
        labels = []
        
        for class_name in self.classes:
            class_dir = os.path.join(base_dir, class_name)
            image_batch = []
            label_batch = []
            
            # Her bir DICOM dosyası için
            for dcm_file in os.listdir(class_dir):
                if dcm_file.endswith('.dcm'):
                    dcm_path = os.path.join(class_dir, dcm_file)
                    print(dcm_path)

                    try:
                        ds_file = pydicom.dcmread(dcm_path)
                        # DICOM dosyasındaki her bir pikseli iterate etme
                        for arr in iter_pixels(ds_file):
                            # Görüntüyü yeniden boyutlandırma
                            resized_image = resize(
                                arr, 
                                (self.img_height, self.img_width), 
                                anti_aliasing=True
                            )
                            
                            # Tek kanallı görüntüyü 3 kanallı hale getirme
                            if len(resized_image.shape) == 2:
                                resized_image_3channel = np.stack((resized_image,)*3, axis=-1)
                            elif resized_image.shape[2] == 1:
                                resized_image_3channel = np.repeat(resized_image, 3, axis=2)
                            else:
                                resized_image_3channel = resized_image
                            
                            # Görüntüyü ekle
                            image_batch.append(resized_image_3channel)
                            
                            # Etiketi ekle
                            label = self.classes.index(class_name)
                            label_batch.append(label)
                        
                    except Exception as e:
                        print(f"Hata oluştu {dcm_path}: {e}")
            
            images.append(image_batch)
            labels.append(label_batch)
                
        return np.array(images), np.array(labels)
    
    def build_model(self):
        """
        VGG16 modelini fine-tune için hazırlama
        
        Returns:
            tensorflow.keras.Model: Hazırlanmış model
        """
        # Temel VGG16 modelini önceden eğitilmiş ağırlıklarla yükleme
        base_model = VGG16(
            weights='imagenet', 
            include_top=False, 
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Temel modelin katmanlarını dondurma
        for layer in base_model.layers:
            layer.trainable = False
        
        # Yeni sınıflandırıcı katmanları ekleme
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation='relu')(x)
        output = Dense(len(self.classes), activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=output)
        
        return model
    
    def compile_model(self, learning_rate=0.0001):
        """
        Modeli derlemek için metod
        
        Args:
            learning_rate (float): Öğrenme oranı
        """
        self.model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
    
    def train(self, base_dir, test_size=0.2, epochs=20, batch_size=32):
        """
        Modeli eğitme metodu
        
        Args:
            base_dir (str): DICOM dosyaları içeren dizin
            test_size (float): Test verisi oranı
            epochs (int): Eğitim epoch sayısı
            batch_size (int): Mini toplu işlem boyutu
        """
        # Görüntüleri ve etiketleri yükleme
        X, y = self.load_dicom_images(base_dir)
        
        # Veriyi eğitim ve test olarak ayırma
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42
        )
        
        # Veriyi ön işleme
        X_train = preprocess_input(X_train)
        X_test = preprocess_input(X_test)
        
        # Fine-tuning için modelin son katmanlarını açma
        for layer in self.model.layers[:15]:
            layer.trainable = False
        for layer in self.model.layers[15:]:
            layer.trainable = True
        
        # Modeli derleme
        self.compile_model()
        
        # Modeli eğitme
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size
        )
        
        return history
    
    def save_model(self, save_path):
        """
        Modeli kaydetme metodu
        
        Args:
            save_path (str): Model kayıt yolu
        """
        self.model.save(save_path)
    
    def plot_training_history(self, history):
        """
        Eğitim geçmişini görselleştirme
        
        Args:
            history: Keras eğitim geçmişi
        """
        plt.figure(figsize=(12, 4))
        
        # Doğruluk grafiği
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
        plt.plot(history.history['val_accuracy'], label='Test Doğruluğu')
        plt.title('Model Doğruluğu')
        plt.xlabel('Epoch')
        plt.ylabel('Doğruluk')
        plt.legend()
        
        # Kayıp grafiği
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Eğitim Kaybı')
        plt.plot(history.history['val_loss'], label='Test Kaybı')
        plt.title('Model Kaybı')
        plt.xlabel('Epoch')
        plt.ylabel('Kayıp')
        plt.legend()
        
        plt.tight_layout()
        plt.show()

# Örnek kullanım
if __name__ == "__main__":
    # DICOM veri dizini
    base_dir = ''
    
    # Modeli oluşturma
    dicom_model = DICOMClassificationModel()
    
    # Modeli eğitme
    history = dicom_model.train(base_dir)
    
    # Eğitim geçmişini görselleştirme
    dicom_model.plot_training_history(history)
    
    # Modeli kaydetme
    dicom_model.save_model('model')

Tcga_lgg_astro_filtered/TCGA-TM-A7CF_1fe1a0ed-c832-4096-a7cc-72b61d4fb592.dcm


TabError: inconsistent use of tabs and spaces in indentation (<string>, line 43)